### Import Required Package

In [85]:
import pandas as pd
import talib

import json

### Import Tidal

In [86]:
import tidal as td

### Initialize Plumber

In [87]:
from pathlib import Path
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

PLUMBER_HOST = "https://dev-api.ddt-dst.cc/api/plumber/"
with open(f'{str(Path.home())}/.config/gcloud/application_default_credentials.json') as plumber_token:
    token = json.load(plumber_token)

### Load market data and compute technical indicators

In [88]:
import pandas as pd
import os
import numpy as np
os.chdir('/Users/jeff/Desktop/individualized-indicator')
test_1 = pd.read_pickle('./emb/2022_12_01/top_7_2023-03-01_2024-06-28.pkl')
df_reset = test_1.reset_index()
unique_values, indices, counts = np.unique(df_reset.iloc[:,0], return_index=True, return_counts=True)
sorted_unique_values = unique_values[np.argsort(indices)]
modified_arr = np.array(sorted_unique_values)
# df_reset.iloc[:,0] = np.array([s[:-3] for s in df_reset.iloc[:,0]]) 
df_reset.iloc[:,0] = np.array(df_reset.iloc[:,0])
multi_index_df = df_reset.set_index(['instrument', 'datetime'])
multi_index_df = multi_index_df.dropna()

DEFAULT_STOCKS = list(modified_arr.astype(object))
start_date = "2023-03-01"
end_date = "2024-06-28"

quote_data = pd.read_parquet(
    f"{PLUMBER_HOST}stocks/tw/ohlcv",
    storage_options={
        "gcp-token": json.dumps(token),
        "start-date": start_date,
        "end-date": end_date,
        "tickers": ",".join([stock for stock in DEFAULT_STOCKS]),
    },
)
quote_data.index.set_levels(
    pd.to_datetime(quote_data.index.levels[1]),
    level=1,
    inplace=True,
)
quote_data.rename_axis(index={
    'ticker': 'instrument'
}, inplace=True)

instruments = quote_data.index.get_level_values('instrument').unique()

quote_data['new'] = multi_index_df.iloc[:,5]

quote_data

/var/folders/2g/rvghmcfx7xvg6mgkvmppv_cw0000gq/T/ipykernel_30794/718369050.py:28: FutureWarning: inplace is deprecated and will be removed in a future version.
  quote_data.index.set_levels(


open      high       low     close    volume  \
instrument datetime                                                       
1101       2023-03-01   37.2023   37.2023   36.0053   36.1968  50523994   
           2023-03-02   36.1968   36.5320   35.8617   36.2926  18212319   
           2023-03-03   36.4841   36.5320   36.0532   36.1489  14693838   
           2023-03-06   36.3883   36.4841   36.2447   36.3405  12088991   
           2023-03-07   36.4362   36.4841   36.2926   36.3405  12798027   
...                         ...       ...       ...       ...       ...   
9958       2024-06-24  336.0000  337.0000  322.0000  329.0000  10111772   
           2024-06-25  330.5000  336.5000  322.0000  325.5000   7282478   
           2024-06-26  326.0000  336.0000  326.0000  329.5000   5833060   
           2024-06-27  327.0000  334.0000  324.5000  325.0000   3860452   
           2024-06-28  325.5000  326.5000  295.0000  302.0000  20309330   

                            new  
instrument datetime              
1101       2023-03-01  0.001477  
           2023-03-02  0.010546  
           2023-03-03  0.002765  
           2023-03-06  0.007385  
           2023-03-07  0.004615  
...                         ...  
9958       2024-06-24  0.001137  
           2024-06-25  0.001369  
           2024-06-26  0.002659  
           2024-06-27  0.001563  
           2024-06-28  0.000645  

[117325 rows x 6 columns]

### Load benchmark data

In [89]:
benchmark_inst = "0050"
benchmark_data = pd.read_parquet(
    f"{PLUMBER_HOST}stocks/tw/ohlcv",
    storage_options={
        "gcp-token": json.dumps(token),
        "start-date": start_date,
        "end-date": end_date,
        "tickers": benchmark_inst,
    },
)
benchmark_data.index.set_levels(
    pd.to_datetime(benchmark_data.index.levels[1]),
    level=1,
    inplace=True,
)
benchmark_data.rename_axis(index={
    'ticker': 'instrument'
}, inplace=True)

benchmark_data

/var/folders/2g/rvghmcfx7xvg6mgkvmppv_cw0000gq/T/ipykernel_30794/2173060872.py:11: FutureWarning: inplace is deprecated and will be removed in a future version.
  benchmark_data.index.set_levels(


open      high       low     close    volume
instrument datetime                                                    
0050       2023-03-01  112.4027  114.4629  111.9715  114.3671   7996077
           2023-03-02  114.3671  114.3671  113.3609  114.2713   3750827
           2023-03-03  114.7983  115.0858  114.2233  114.3671   3341290
           2023-03-06  115.1337  115.8524  114.8941  115.4691   9805282
           2023-03-07  115.5170  116.3315  115.1816  116.2357   7894646
...                         ...       ...       ...       ...       ...
           2024-06-24  185.5518  185.5518  182.6168  183.0645  14671365
           2024-06-25  181.0250  182.9651  179.8808  182.9651  13774562
           2024-06-26  184.0097  185.1539  183.3133  184.4077   9269483
           2024-06-27  182.6666  184.3579  182.4676  184.3579   6508242
           2024-06-28  184.3579  186.4473  183.7112  185.5021   4455951

[325 rows x 5 columns]

### Initialize Tidal
1. Initialize Tidal object
2. Add Quote data (pd.DataFrame)
3. Set strategy object (td.BaseStrategy)
4. Add metric objects (td.BaseMetic)

In [90]:
# Initialize Tidal object
tidal = td.Tidal(init_cash=1000000, slip_ticks=1, stock_config=td.StockConfig.TW, load_configs=True, reqMem="1000Mi")

# Add quote data
tidal.add_quote(quote_data)

# Set strategy object
tidal.set_strategy(td.strategy.TopkDropout(5, 2, 'new'))

# Set metric objects
tidal.add_metric(td.metric.AccountInfo())
tidal.add_metric(td.metric.AdditionalInfo())
tidal.add_metric(td.metric.PositionInfo())
tidal.add_metric(td.metric.Portfolio(benchmark_data.loc[benchmark_inst]))

### Stock Config

In [91]:
tidal.exchange.stock_config

InstConfig {Margin:0.0, Tick Size:0.002, Tick Value:0.002, Trade Unit:1000, Commission:0.0004275, Min Commission:20.0, Transaction Tax:0.003}

### Instrument Configs

In [92]:
tidal.exchange.inst_configs

{'MTX': InstConfig {Margin:46000.0, Tick Size:1.0, Tick Value:50.0, Trade Unit:1, Commission:20.0, Min Commission:0.0, Transaction Tax:2e-05},
 'NQ': InstConfig {Margin:16500.0, Tick Size:0.25, Tick Value:5.0, Trade Unit:1, Commission:1.85, Min Commission:0.0, Transaction Tax:0.0},
 'TX': InstConfig {Margin:184000.0, Tick Size:1.0, Tick Value:200.0, Trade Unit:1, Commission:40.0, Min Commission:0.0, Transaction Tax:2e-05}}

### Config Modification

In [93]:
# Set commission to 77% off
tidal.exchange.set_stock_config(commission=0.001425 * 0.23)
tidal.exchange.stock_config

InstConfig {Margin:0.0, Tick Size:0.002, Tick Value:0.002, Trade Unit:1000, Commission:0.00032775, Min Commission:20.0, Transaction Tax:0.003}

### Add New Instrument Config

In [94]:
# Add MGC config
tidal.exchange.set_config(instrument='MGC', margin=787., tick_size=0.1, tick_value=1., trade_unit=1, commission=2., min_commission=0., transaction_tax=0.)
tidal.exchange.inst_configs

{'MTX': InstConfig {Margin:46000.0, Tick Size:1.0, Tick Value:50.0, Trade Unit:1, Commission:20.0, Min Commission:0.0, Transaction Tax:2e-05},
 'NQ': InstConfig {Margin:16500.0, Tick Size:0.25, Tick Value:5.0, Trade Unit:1, Commission:1.85, Min Commission:0.0, Transaction Tax:0.0},
 'TX': InstConfig {Margin:184000.0, Tick Size:1.0, Tick Value:200.0, Trade Unit:1, Commission:40.0, Min Commission:0.0, Transaction Tax:2e-05},
 'MGC': InstConfig {Margin:787.0, Tick Size:0.1, Tick Value:1.0, Trade Unit:1, Commission:2.0, Min Commission:0.0, Transaction Tax:0.0}}

### Start Backtesting

In [95]:
tidal.backtest()

Tidal Backtesting: 100%|██████████| 325/325 [02:54<00:00,  1.86it/s, cash=1.23e+5, pnl=5.14e+5, position_cost=9.3e+5, value=1.57e+6] 


### Metric - AccountInfo

In [96]:
tidal.metrics['AccountInfo'].report

,cash,trade_cost,position_cost,slip_cost,pnl,value,max_drawdown,turnover
datetime,,,,,,,,
2023-03-01,1000000.000000,0.000000,0.000000e+00,0.0000,0.0000,1.000000e+06,0.000000,0.0000
2023-03-02,270677.786963,238.957037,7.290833e+05,6358.2560,1790.3440,1.001551e+06,0.000000,729083.2560
2023-03-03,66226.046260,305.944140,9.334680e+05,8524.6096,2303.3904,1.001997e+06,0.000000,204384.7536
2023-03-06,66226.046260,305.944140,9.334680e+05,8524.6096,21329.5904,1.021024e+06,0.000000,0.0000
2023-03-07,66226.046260,305.944140,9.334680e+05,8524.6096,42815.8904,1.042510e+06,0.000000,0.0000
...,...,...,...,...,...,...,...,...
2024-06-24,12144.857233,98038.382967,1.031999e+06,87710.6598,482631.4664,1.526775e+06,-0.038010,0.0000
2024-06-25,118791.818336,98894.285464,9.389364e+05,87715.2962,475226.5288,1.532955e+06,-0.034116,380056.4636
2024-06-26,116594.973173,99386.756027,9.389316e+05,88297.2708,502063.7288,1.557590e+06,-0.018594,270839.6254


### Metric - PositionInfo

In [97]:
tidal.metrics['PositionInfo'].report

quantity      price  commission  slip_cost        pnl  \
instrument datetime                                                            
1101       2023-08-22    5000.0  34.510500   56.554082   728.0000  -728.0000   
           2023-11-22    5000.0  33.120509   54.276234  -397.4560 -1301.5440   
2233       2023-03-02    2000.0  95.511842   62.608013  1289.0848   526.5152   
           2023-03-03    2000.0  95.511842   62.608013  1289.0848  3249.9152   
           2023-03-06    2000.0  95.511842   62.608013  1289.0848  9604.7152   
...                         ...        ...         ...        ...        ...   
8091       2024-01-11    2000.0  84.473911   55.372648   894.4212  -523.0212   
           2024-01-23    2000.0  85.698000   56.175039   185.8000  -185.8000   
           2024-01-26    2000.0  88.009167   57.690009   722.7340 -2765.5340   
           2024-01-29    2000.0  88.009167   57.690009   722.7340 -1651.3340   
9911       2023-04-26    3000.0  57.691252   56.724924   345.4566  1008.1434   

                       now_price  
instrument datetime               
1101       2023-08-22    34.3649  
           2023-11-22    32.8602  
2233       2023-03-02    95.7751  
           2023-03-03    97.1368  
           2023-03-06   100.3142  
...                          ...  
8091       2024-01-11    84.2124  
           2024-01-23    85.6051  
           2024-01-26    86.6264  
           2024-01-29    87.1835  
9911       2023-04-26    58.0273  

[1618 rows x 6 columns]

In [98]:
pi_report = tidal.metrics['PositionInfo'].report
pi_report.iloc[pi_report.index.get_level_values('datetime') == '2022-06-14']

,,quantity,price,commission,slip_cost,pnl,now_price
instrument,datetime,,,,,,


### Metric - AdditionalInfo

In [99]:
tidal.metrics['AdditionalInfo'].report

new
instrument datetime            
1101       2023-03-01  0.001477
           2023-03-02  0.010546
           2023-03-03  0.002765
           2023-03-06  0.007385
           2023-03-07  0.004615
...                         ...
9958       2024-06-24  0.001137
           2024-06-25  0.001369
           2024-06-26  0.002659
           2024-06-27  0.001563
           2024-06-28  0.000645

[117325 rows x 1 columns]

### Trade Report

In [100]:
tidal.trade_report

,win_num,lose_num,trade_num,pos_num,win_rate,profit,loss,trade_cost,pnl
instrument,,,,,,,,,
6664,1.0,0.0,1.0,1.0,1.000000,293818.452800,0.000000,188850.547200,293818.452800
2404,1.0,0.0,1.0,1.0,1.000000,141009.446400,0.000000,204384.753600,141009.446400
5284,3.0,2.0,5.0,1.0,0.600000,96281.362916,-7460.754622,130858.049106,88820.608294
5443,8.0,10.0,18.0,1.0,0.444444,159143.074279,-70873.403809,256544.335930,88269.670470
6143,12.0,28.0,40.0,0.0,0.300000,95154.353727,-81106.934639,26747.455512,14047.419088
3617,1.0,1.0,2.0,0.0,0.500000,10720.735063,-47.826502,1440.100639,10672.908561
4549,1.0,1.0,2.0,0.0,0.500000,10128.610157,-2067.958607,1473.504850,8060.651550
2439,1.0,0.0,1.0,0.0,1.000000,2089.337903,0.000000,1010.262097,2089.337903
9911,1.0,0.0,1.0,0.0,1.000000,25.112236,0.000000,634.867364,25.112236


### Traded instruments

In [101]:
tidal.account.trades.keys()

dict_keys(['5284', '2404', '6470', '6664', '2233', '6147', '8049', '6143', '8091', '3416', '6591', '6523', '9911', '2548', '3594', '5443', '3617', '1101', '3669', '2439', '4549', '2542'])

### Trading History

In [102]:
# tidal.account.trades['3015']

### Plot chart by using Plotly

In [103]:
# tidal.analyzer.inst_chart(instrument='3015', metric_name='AdditionalInfo', plot_type=td.PlotType.LINE, scale=1.0)

### Tidal Dashboard

In [104]:
tidal.tdboard()

 * Serving Flask app 'tidal.tdboard'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:61852
 * Running on http://192.168.199.13:61852
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:192.168.199.13 - - [21/Aug/2024 21:53:48] "GET / HTTP/1.1" 200 -
INFO:werkzeug:192.168.199.13 - - [21/Aug/2024 21:53:48] "GET /static/js/main.d754b0a3.js HTTP/1.1" 200 -
INFO:werkzeug:192.168.199.13 - - [21/Aug/2024 21:53:49] "GET /static/css/main.bf4d504b.css HTTP/1.1" 200 -
INFO:werkzeug:192.168.199.13 - - [21/Aug/2024 21:53:49] "GET /images/Tidal_Logo_white.png HTTP/1.1" 200 -
INFO:werkzeug:192.168.199.13 - - [21/Aug/2024 21:53:51] "GET /api/quote/inst_list HTTP/1.1" 200 -
INFO:werkzeug:192.168.199.13 - - [21/Aug/2024 21:53:51] "GET /api/metric/metric_list HTTP/1.1" 200 -
INFO:werkzeug:192.168.199.13 - - [21/Aug/2024 21:53:51] "GET /api/trade/trade_report HTTP/1.1" 200 -
INFO:werkzeug:192.